# Create the grid for Richards 1D

This code allows to create a mesh for 1D PDE problem:
    - domain discretization
    - parameters setting
    - initial condition setting
All output data are stored in a NetCDF file

    -Author: Niccolò Tubini and Riccardo Rigon
    -License: Creative Commons 4.0

In [1]:
%%capture
%run RichardsMeshGenBrooksCorey.py

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from netCDF4 import Dataset

In [3]:
# Standard imports 
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import BoxSelectTool
from bokeh.models import HoverTool

output_notebook()

Loading BokehJS ...

## Define some variables:
- `folderPath`: path of the fodlder containing the input file (.csv) -> your_path/data/RichardMeshGen_input
- `inFileName`: name of the input file (.csv)


- `outFileName`: name of the output file (.nc) with the entire file path -> your_path/data/Grid_NetCDF/grid.nc
- `outputTitle`: string containing a title for the output, it is saved within the file
- `outputSummary`: string containing a description for the output, it is saved within the file (global metadata)
- `outputDate`: string containing the date of file creation, optionally
- `outputInstitution`: string containing the name of whom created the file, optionally

In [5]:
folderPath = "../data/RichardMeshGen_input"

inputFileName = "Clay_noPonding_BC.csv"

outputFileName =  "C:/Users/Niccolo/Desktop/OMS_Project_Richards1D/data/Grid_NetCDF/Clay_BC.nc"

outputTitle = '''

                '''
outputSummary = '''
        Type,eta,N,psi,thetaS,thetaR,Ks,n,psiD,alphaSpecificStorage,betaSpecificStorage,et
        L,0,40,-2,0.35,0.07,0.0000981,1.5,-0.3496,0,0,0
        L,-2,-999,0,-999,-999,-999,-999,-999,-999,-999,-999
'''

outputDate = '24/09/2018'

outputInstitution = 'Geoframe'

## Define the type of intial condition for water suction $\psi$:
Possible choices are:

- `hydrostatic`
- `constant`
- `linear interpolation`

In [6]:
icType = 'hydrostatic'

## Set some plot features

In [7]:
labelSize = 18
titleSize = 22
legendSize = 14
axisTicksSize = 14

lineWidth =0.3

In [8]:
os.chdir(folderPath)

In [9]:
data = pd.read_csv(inputFileName)
#print(data.head())
#print(data.index)
#print(data.columns)
data

,Type,eta,N,psi,thetaS,thetaR,Ks,n,psiD,alphaSpecificStorage,betaSpecificStorage,et
0,L,0,40,-2,0.35,0.07,0.000098,1.5,-0.3496,0,0,0
1,L,-2,-999,0,-999.00,-999.00,-999.000000,-999.0,-999.0000,-999,-999,-999


### This section computes geometrical quantities of the discretized domain
For an explanantion of the data in the above dataFrame, please read first the presentation at 

### From the data in the csv file, the main numpy array are built:
- `eta`: vertical coordinate of control volume centroids, positive upward with origin set at soil surface [m]
- `etaDual`: vertical coordinate of control volume interfaces, positive upward with origin set at soil surface [m]
- `deltaZ`: length of each control volume [m]
- `spaceDelta`: distance between two adjacent control volume. It is used to compute gradients [m]
- `z`: vertical coordinate of control volume centroids, positive upward with origin set at soil column bottom [m]
- `zDual`: vertical coordinate of control volume interfaces, positive upward with origin set at soil column bottom [m]

In [10]:
[eta,etaDual,deltaZ,spaceDelta,z,zDual]=buildData(data)

In [11]:
showMesh(data,z,eta,etaDual,labelSize,titleSize,legendSize,axisTicksSize,lineWidth)

### From the data in the csv file, the initial condition for water suction $\psi$ is defined
- hydrostatic distribution for a given $\psi$ @bottom -> 'hydrostatic'
- constant distribution for a given $\psi$ @bottom -> 'constant'
- linear interpolation from measurement points -> 'linear interpolation'

In [12]:
psiIC = setInitialCondition(data,eta,z,icType)

In [13]:
showInitialCondition(psiIC,eta,icType,labelSize,titleSize,legendSize,axisTicksSize,lineWidth)

In [14]:
showTotalHead(psiIC,z,eta,icType,labelSize,titleSize,legendSize,axisTicksSize,lineWidth)

### Assign hydraulic properties to each control volume of the grid
For an explanantion of the data in the above dataFrame, please read first the presentation at 

In [15]:
[thetaS, thetaR, Ks, alphaSpecificStorage, betaSpecificStorage, n, psiD, par3SWRC, par4SWRC, par5SWRC, psiStar, par7SWRC, par8SWRC, et] = setParameters(data,eta)

In [16]:
showParameters(thetaS,thetaR,Ks,alphaSpecificStorage,betaSpecificStorage,n,psiD,psiStar,et,eta,labelSize,titleSize,axisTicksSize,lineWidth)


## Save all grid data in a NetCDF file


In [17]:
writeGridNetCDF(eta,etaDual,z,zDual,spaceDelta,deltaZ,psiIC,thetaS,thetaR,Ks,alphaSpecificStorage,betaSpecificStorage,n,psiD,par3SWRC,par4SWRC,par5SWRC,psiStar,par7SWRC,par8SWRC,et,outputFileName,outputTitle,outputInstitution,outputSummary,folderPath,outputDate,inputFileName)

*** SUCCESS writing!  C:/Users/Niccolo/Desktop/OMS_Project_Richards1D/data/Grid_NetCDF/Clay_BC.nc
